# Clip

## Instalações de pacotes

In [ ]:
# !pip install torch torchvision transformers datasets tqdm
# !pip install datasets[image] pillow

## Imports iniciais

In [ ]:
import os
from zipfile import ZipFile, BadZipFile
import random
import pickle
import heapq
import json
from collections import defaultdict

import torch
from transformers import CLIPModel, CLIPProcessor
from PIL import Image
from tqdm import tqdm

## Dataset COCO

In [ ]:
!wget http://images.cocodataset.org/zips/train2017.zip -O coco_train2017.zip
!wget http://images.cocodataset.org/zips/val2017.zip -O coco_val2017.zip
!wget http://images.cocodataset.org/annotations/annotations_trainval2017.zip -O coco_ann2017.zip

In [ ]:
# 2. Método para extrair arquivos ZIP
def extrair_arquivo(caminho):
    try:
        with ZipFile(caminho + ".zip") as zipf:
            zipf.extractall(caminho)
        os.remove(caminho + ".zip")
    except BadZipFile as e:
        print("Erro ao extrair:", e)

# Extrair dados do COCO
extrair_arquivo("./coco_train2017")
extrair_arquivo("./coco_val2017")
extrair_arquivo("./coco_ann2017")

## Carregando modelo CLIP

In [ ]:
# 3. Configurações iniciais
dispositivo = "cuda" if torch.cuda.is_available() else "cpu"
diretorio_val = "./coco_val2017/val2017"
caminho_anotacoes = "./coco_ann2017/annotations/captions_val2017.json"

# 4. Carregar modelo CLIP
modelo = CLIPModel.from_pretrained("openai/clip-vit-base-patch32").to(dispositivo)
processador = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")
modelo.eval()

## Organização dos dados

In [ ]:
# Precomputar embeddings das imagens de validação
caminhos_imagens = [
    os.path.join(diretorio_val, f)
    for f in os.listdir(diretorio_val) if f.endswith(".jpg")
]
embeddings_imagem = {}
for caminho in tqdm(caminhos_imagens, desc="Pré-computando embeddings"):
    img = Image.open(caminho).convert("RGB")
    inputs = processador(images=img, return_tensors="pt").to(dispositivo)
    with torch.no_grad():
        emb = modelo.get_image_features(**inputs)
        emb = emb / emb.norm(dim=-1, keepdim=True)
    img_id = int(os.path.basename(caminho).split(".")[0])
    embeddings_imagem[img_id] = emb.cpu()

# Salvar embeddings em arquivo para reutilização
with open("embeddings_val.pkl", "wb") as f:
    pickle.dump(embeddings_imagem, f)

# Carregar legendas e organizar por ID de imagem
with open(caminho_anotacoes, 'r') as f:
    anotacoes = json.load(f)
legendas_por_imagem = defaultdict(list)
for entry in anotacoes['annotations']:
    img_id = entry['image_id']
    legendas_por_imagem[img_id].append(entry['caption'])

## Funções auxiliares

In [ ]:
# Função para criar exemplos com mistura de distractores difíceis e fáceis
def criar_exemplo_calibrado(
    id_alvo, embeddings, modelo, processador,
    k=9, tamanho_pool=2000, n_facil=7
):
    # Selecionar pool de imagens
    pool = random.sample(list(embeddings.keys()), tamanho_pool)
    pool = [i for i in pool if i != id_alvo]
    # Escolher legenda
    legenda = random.choice(legendas_por_imagem[id_alvo])
    # Computar embedding de texto
    inputs = processador(text=[legenda], return_tensors="pt", padding=True).to(dispositivo)
    with torch.no_grad():
        emb_texto = modelo.get_text_features(**inputs)
        emb_texto = emb_texto / emb_texto.norm(dim=-1, keepdim=True)
        emb_texto = emb_texto.cpu()
    # Calcular similaridade com imagens no pool
    sims = [(float((emb_texto @ embeddings[i].T)), i) for i in pool]
    # Selecionar distractores difíceis
    top_duros = [i for _, i in heapq.nlargest(k - n_facil, sims, key=lambda x: x[0])]
    # Adicionar distractores fáceis
    faciles = random.sample([i for i in pool if i not in top_duros], n_facil)
    distractores = top_duros + faciles
    return {
        "legenda": legenda,
        "id_correto": id_alvo,
        "ids_distratores": distractores
    }

# Função para calcular Recall@1
def calcular_recall_1(exemplo, modelo, processador):
    # Preparar imagens
    ids = [exemplo["id_correto"]] + exemplo["ids_distratores"]
    imagens = [
        Image.open(f"{diretorio_val}/{str(i).zfill(12)}.jpg") for i in ids
    ]
    # Processar entrada CLIP
    inputs = processador(text=[exemplo["legenda"]], images=imagens,
                         return_tensors="pt", padding=True).to(dispositivo)
    with torch.no_grad():
        saida = modelo(**inputs)
        logits = saida.logits_per_text.cpu().numpy()[0]
    # Verificar top-1
    indice_top1 = logits.argmax()
    return int(ids[indice_top1] == exemplo["id_correto"])

## Calculando recall 1

In [ ]:
# Carregar embeddings do arquivo
with open("embeddings_val.pkl", "rb") as f:
    embeddings_imagem = pickle.load(f)

# Gerar exemplos calibrados e avaliar
N = 2000
exemplos = [
    criar_exemplo_calibrado(
        id_alvo=random.choice(list(legendas_por_imagem.keys())),
        embeddings=embeddings_imagem,
        modelo=modelo,
        processador=processador,
        k=9,
        tamanho_pool=2000,
        n_facil=7
    )
    for _ in range(N)
]

# Calcular recall geral
acertos = sum(calcular_recall_1(ex, modelo, processador) for ex in tqdm(exemplos))
print(f"Recall@1 calibrado: {acertos / N:.2%}")

## Analisando os erros

In [ ]:
import random, numpy as np, torch
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
if torch.cuda.is_available(): torch.cuda.manual_seed_all(SEED)


def calcular_recall_k(exemplo, modelo, processador, k=5):
    ids = [exemplo["id_correto"]] + exemplo["ids_distratores"]
    imagens = [Image.open(f"{diretorio_val}/{str(i).zfill(12)}.jpg") for i in ids]
    inputs = processador(text=[exemplo["legenda"]], images=imagens,
                         return_tensors="pt", padding=True).to(dispositivo)
    with torch.no_grad():
        logits = modelo(**inputs).logits_per_text.cpu().numpy()[0]
    topk = logits.argsort()[-k:][::-1]
    return int(exemplo["id_correto"] in [ids[idx] for idx in topk])

def calcular_mrr(exemplo, modelo, processador):
    ids = [exemplo["id_correto"]] + exemplo["ids_distratores"]
    imagens = [Image.open(f"{diretorio_val}/{str(i).zfill(12)}.jpg") for i in ids]
    inputs = processador(text=[exemplo["legenda"]], images=imagens,
                         return_tensors="pt", padding=True).to(dispositivo)
    with torch.no_grad():
        logits = modelo(**inputs).logits_per_text.cpu().numpy()[0]
    ranks = logits.argsort()[::-1]
    rank_pos = list(ranks).index(0) + 1
    return 1.0 / rank_pos


import pandas as pd

regs = []
for ex in tqdm(exemplos, desc="Rodando avaliação detalhada"):
    regs.append({
        "legenda": ex["legenda"],
        "hit@1": calcular_recall_1(ex, modelo, processador),
        "hit@5": calcular_recall_k(ex, modelo, processador, k=5),
        "mrr":   calcular_mrr(ex, modelo, processador)
    })
df = pd.DataFrame(regs)
print("Recall@1:", df["hit@1"].mean(),
      "Recall@5:", df["hit@5"].mean(),
      "MRR:", df["mrr"].mean())


In [ ]:
df["num_tokens"]    = df["legenda"].apply(lambda t: len(t.split()))
df["tamanho_chars"] = df["legenda"].str.len()

# Função de classificação de erro
import re

def classificar_erro(legenda):
    if len(legenda) < 3:
        return "muito_curta"
    if len(legenda) > 50:
        return "muito_longa"
    if re.search(r"\b(e|and|,)\b", legenda):
        return "ambigua"
    return "outros"

df_fail = df[df["hit@1"] == 0].copy()
df_fail["tipo_erro"] = df_fail["legenda"].apply(classificar_erro)


# Análise de distribuições
print("Distribuição de erros:")
print(df_fail["tipo_erro"].value_counts(normalize=True))

# visualizar contagens absolutas também
print(df_fail["tipo_erro"].value_counts())


# Explorar exemplos de cada classe
for categoria in df_fail["tipo_erro"].unique():
    print(f"\n=== Exemplos de {categoria} ===")
    display(df_fail[df_fail["tipo_erro"] == categoria].head(5)[["legenda"]])

In [ ]:
import matplotlib.pyplot as plt

vc = df_fail["tipo_erro"].value_counts(normalize=True)

plt.figure(figsize=(6,4))
vc.plot.bar()
plt.title("Distribuição de tipos de erro (Recall@1 falhos)")
plt.ylabel("Proporção")
plt.xlabel("Tipo de erro")
plt.tight_layout()
plt.show()

top3 = vc.head(3)
print("Top-3 categorias de erro:")
for cat, pct in top3.items():
    print(f"  • {cat}: {pct:.1%}")

for cat in top3.index:
    print(f"\n=== Exemplos de {cat} ===")
    display(df_fail[df_fail["tipo_erro"] == cat].head(3)[["legenda"]])


In [ ]:
#Filtrar só os erros “outros”
df_outros = df_fail[df_fail["tipo_erro"] == "outros"].copy()

print("Estatísticas de palavras (outros):")
print(df_outros["num_tokens"].describe(), "\n")
print("Estatísticas de caracteres (outros):")
print(df_outros["tamanho_chars"].describe(), "\n")

print("===== 10 exemplos aleatórios de 'outros' =====")
for leg in df_outros["legenda"].sample(10, random_state=42).tolist():
    print(" •", leg)
print("\n")

from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(
    ngram_range=(1,2),
    max_features=20,
    stop_words=None
)
X = vectorizer.fit_transform(df_outros["legenda"])
freqs = X.toarray().sum(axis=0)
ngrams = vectorizer.get_feature_names_out()

print("Top 20 uni-/bi-grams mais comuns em 'outros':")
for ng, f in sorted(zip(ngrams, freqs), key=lambda x: x[1], reverse=True):
    print(f"{ng}: {f}")


In [ ]:
import random
import matplotlib.pyplot as plt
from PIL import Image

errs = []
for ex, hit in zip(exemplos, df["hit@1"]):
    if hit == 0:
        errs.append({"id": ex["id_correto"], "legenda": ex["legenda"]})

N = 9
sample = random.sample(errs, N)

fig, axes = plt.subplots(3, 3, figsize=(12,12))
for ax, item in zip(axes.flatten(), sample):
    img_id = item["id"]
    caption = item["legenda"]
    path = f"{diretorio_val}/{str(img_id).zfill(12)}.jpg"
    img = Image.open(path)
    ax.imshow(img)
    ax.set_title(caption, fontsize=10)
    ax.axis("off")

plt.suptitle("Exemplos de imagens mal recuperadas (Recall@1 até 10 opções)", fontsize=14)
plt.tight_layout(rect=[0, 0, 1, 0.96])
plt.show()


In [ ]:
import random
import pandas as pd

N = 50

legendas_amb = df_fail[df_fail["tipo_erro"] == "ambigua"]["legenda"] \
                  .sample(N, random_state=42) \
                  .reset_index(drop=True)

legendas_out = df_fail[df_fail["tipo_erro"] == "outros"]["legenda"] \
                  .sample(N, random_state=42) \
                  .reset_index(drop=True)

df_compare = pd.DataFrame({
    "Ambíguas": legendas_amb,
    "Outros":   legendas_out
})

display(df_compare)

for i in range(N):
    print(f"{i+1:2d}. A: {df_compare.loc[i, 'Ambíguas']}")
    print(f"    B: {df_compare.loc[i, 'Outros']}\n")
